[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/earthpulse/pytorch_eo/blob/master/challenges/euroavia_hackathon_21/solution.ipynb)

> If you run this notebook on google colab, uncomment the following cell to install the required dependencies

In [1]:
# !pip install pytorch_eo timm

In [2]:
import timm
import torch
from einops import rearrange

class Model(torch.nn.Module):
    def __init__(self, backbone, in_chans, pretrained=None, num_classes=10, max_value=4000):
        super().__init__()
        # download resnet18
        self.model = timm.create_model(
            backbone,
            pretrained=pretrained,
            in_chans=in_chans,
            num_classes=num_classes
        )
        self.max_value = max_value

    def forward(self, x):
        x = rearrange(x, 'b h w c -> b c h w')
        x = x / self.max_value
        x = x.clip(0, 1)
        return self.model(x)

In [3]:
import pytorch_lightning as pl
from pytorch_eo.tasks.classification import ImageClassification
from pytorch_eo.datasets.eurosat import EuroSAT
from pytorch_eo.utils.sensors import S2
import albumentations as A

pl.seed_everything(42, workers=True) 

trans = A.Compose([
    A.Transpose(),
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.RandomRotate90()
])

ds = EuroSAT(batch_size=256, bands=S2.RGB, val_size=0., num_workers=20, pin_memory=True) 

model = Model(backbone='seresnext50_32x4d', in_chans=3, pretrained='imagenet') 

hparams = {
    'optimizer': 'Adam',
    'optim_params': {
        'lr': 1e-3,
    },
    'scheduler': 'MultiStepLR',
    'scheduler_params': {
        'milestones': [3, 6],
        'gamma': 0.1,
    }
}

task = ImageClassification(model, hparams=hparams)

trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=10,
    deterministic=True 
)

trainer.fit(task, ds)

Global seed set to 42
Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | Model            | 25.5 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
25.5 M    Trainable params
0         Non-trainable params
25.5 M    Total params
102.126   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/juan/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:382: UserWarning: One of given dataloaders is None and it will be skipped.
  rank_zero_warn("One of given dataloaders is None and it will be skipped.")
Global seed set to 42


Training: -1it [00:00, ?it/s]

In [4]:
trainer.test()

/home/juan/miniconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9835185408592224, 'test_loss': 0.05512882396578789}
--------------------------------------------------------------------------------


[{'test_loss': 0.05512882396578789, 'test_acc': 0.9835185408592224}]